In [27]:
# data cleaning
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="myApp")


def create_labels(n):
    labels = []
    for i in range(0, len(n)-1):
        if i < len(n) -1:
            labels.append(str(n[i]) + " and " + str(n[i + 1]))
    return labels

In [28]:
speed_data = pd.read_csv("../data/raw/Speed_Dating.csv",encoding='latin')

In [29]:
speed_data.head()
speed_data.shape

(8378, 195)

In [30]:
# transform to get one row per couple
speed_data["date_id"] = [f"{a}-{b}-{c}-{d}" for a,b,c,d in zip(speed_data.wave, speed_data.order, speed_data["round"], speed_data.position)]

In [31]:
# features

identifiers = ["date_id", "iid", "match", "wave"]

#preference is that partners preference which is covered when we pivot the table to have one date per row
#preference = ["pf_o_att", "pf_o_sin", "pf_o_int", "pf_o_fun", "pf_o_amb", "pf_o_sha"]

personal_info = ["field_cd", "age", "race", "income", "zipcode", "samerace", "goal", "from", "gender"]

behaviour = ["go_out",
           "sports", "tvsports", "exercise", "dining", "museums", "art", "hiking", "gaming",
           "clubbing", "reading", "tv", "theater", "movies", "concerts", "music", "shopping",
           "yoga", "exphappy"]

looking_for = ["attr1_1", "sinc1_1", "intel1_1", "fun1_1", "amb1_1", "imprace", "imprelig"]

self_rating = [ "attr3_1", "sinc3_1", "intel3_1", "fun3_1", "amb3_1"]

to_impute = behaviour + looking_for + self_rating + ["field_cd", "age", "goal"]

to_normalise = behaviour + looking_for + self_rating


In [32]:
# difference between what person b i slooking for a what person a rates themselves 1_1_b and 3_1_a
# difference between what person a i slooking for b what person a rates themselves 1_1_a and 3_1_b

In [33]:
speed_data = speed_data[identifiers  + personal_info + behaviour + looking_for + self_rating]#+ preference

In [34]:
for i in speed_data[to_impute]:
 speed_data.loc[speed_data.loc[:,i].isnull(),i]=speed_data.loc[:,i].mean()

In [35]:
speed_data["from"] = speed_data["from"].replace("new york city", "New York")

df_ny_zip = speed_data[speed_data["from"] == "New York"]["zipcode"]
ny_zip = df_ny_zip.loc[df_ny_zip.index[0]]
print("A New York zipcode is: ", ny_zip)

df_nj_zip = speed_data[speed_data["from"] == "New Jersey"]["zipcode"]
nj_zip = df_nj_zip.loc[df_nj_zip.index[0]]
print("A New Jersey zipcode is: ", nj_zip)

m1 = speed_data['zipcode'].isnull()
m2 = speed_data["from"] == "New Jersey"
speed_data.loc[m1 & m2, 'zipcode'] = nj_zip 

m1 = speed_data['zipcode'].isnull()
m2 = speed_data["from"] == "New York"
speed_data.loc[m1 & m2, 'zipcode'] = ny_zip 

speed_data["zipcode"] = speed_data["zipcode"].fillna("Not applicable")

A New York zipcode is:  10,028
A New Jersey zipcode is:  7,661


In [36]:
# feature engineering
speed_data["income"] = speed_data["income"].str.replace(",", "")

speed_data["income"] = speed_data["income"].astype("float")
speed_data["income"].fillna(-1, inplace=True)
bins = np.arange(speed_data["income"].min()-1, speed_data["income"].max(), 10000)
labels = create_labels(bins)
labels[0] = "Not provided"
speed_data["income"] = pd.cut(x=speed_data["income"],bins = bins, labels = labels)
speed_data["income"]

speed_data["zipcode"] = speed_data["zipcode"].astype("str").str.replace(",", "")
speed_data["zipcode_area"] = [i[0:1] for i in speed_data["zipcode"]]
#speed_data.drop("zipcode", axis = 1, inplace = True)

In [37]:
#speed_data_no_null = speed_data[speed_data['zipcode'].str.isnumeric()]


In [38]:
# from geopy.geocoders import Nominatim
# import geopy
# geopy.geocoders.options.default_user_agent = "my-application"
# geolocator = Nominatim(user_agent="my_user_agent")

In [39]:
# lat_list = []
# long_list = []

# for x in speed_data["zipcode"]:
#     try:
#         location = geolocator.geocode(x)
#         lat = location.latitude
#         long = location.longitude
#     except:
#         lat = "Invalid postcode"
#         long = "Invalid postcode"
        
#     lat_list.append(lat)
#     long_list.append(long)

In [40]:
# speed_data["latitude"] = lat_list
# speed_data["longitude"] = long_list
# speed_data.to_csv("speed_data_coords.csv")

In [41]:
coords_df = pd.read_csv("../data/interim/speed_data_coords.csv")
coords_df = coords_df[["zipcode", "latitude", "longitude"]]
coords_df = coords_df.drop_duplicates()

In [42]:
speed_data = speed_data.merge(coords_df, on="zipcode", how = "left")

In [43]:
speed_data["var_undergrad"] = [1 if x == 4 else 0 for x in speed_data["wave"]]
speed_data["var_budget"] = [1 if x == 12 else 0 for x in speed_data["wave"]]
speed_data["var_diff_mc"] = [1 if x == 13 else 1 if x == 14 else 0 for x in speed_data["wave"]]
speed_data["var_mag"] = [1 if x == 18 else 1 if x == 21 else 0 for x in speed_data["wave"]]
speed_data["var_book"] = [1 if x == 19 else 1 if x == 20 else 0 for x in speed_data["wave"]]

In [44]:
# normalise continuous variables to be between 0 and 1
speed_data[to_normalise] = speed_data[to_normalise].apply(lambda x: (x-x.min())/ (x.max()-x.min()), axis=0)

In [45]:
speed_data["sports_person"] = (speed_data["sports"] + speed_data["exercise"] + speed_data["tvsports"])/3
speed_data["creative_person"] = (speed_data["theater"] + speed_data["movies"] + speed_data["concerts"] + speed_data["music"] + speed_data["art"])/5

speed_data["creative_person"].quantile([0.75])
speed_data["sports_person"].quantile([0.75])

def sports_thresh(value):
    if value > 0.704:
        return 1
    else:
        return 0
 
speed_data['is_sporty'] = speed_data['sports_person'].map(sports_thresh)

def sports_thresh(value):
    if value > 0.835:
        return 1
    else:
        return 0
 
speed_data['is_creative'] = speed_data['creative_person'].map(sports_thresh)

In [46]:
speed_data.to_csv("../data/interim/clean_data_multi_row.csv")

In [47]:
count = 0

for d in speed_data.date_id.unique():
  
    if count == 0:
        speed_data_date1 = speed_data[speed_data["date_id"] == d]
        if len(speed_data_date1) == 2:
            speed_data_date1.iid = ["a", "b"]
        else:
            #print(speed_data1["date_id"])
            next
    
    if count != 0:
        speed_data_date2 = speed_data[speed_data["date_id"] == d]
        if len(speed_data_date2) == 2:
            speed_data_date2.iid = ["a", "b"]
        else:
            #print(speed_data1["date_id"])
            next    
        speed_data_date1 = pd.concat([speed_data_date1, speed_data_date2])

    count = count + 1

C:\Users\hannah.alexander\AppData\Local\Temp\ipykernel_10668\1931704513.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speed_data_date1.iid = ["a", "b"]
C:\Users\hannah.alexander\AppData\Local\Temp\ipykernel_10668\1931704513.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speed_data_date2.iid = ["a", "b"]
C:\Users\hannah.alexander\AppData\Local\Temp\ipykernel_10668\1931704513.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [48]:
#pd.unique(speed_data_date1.iid)
# TODO why dont all date_ids have 2 entries?

In [49]:
df = speed_data_date1[speed_data_date1["iid"] == "a"]
speed_data_date_a = df.add_suffix("_a")

df = speed_data_date1[speed_data_date1["iid"] == "b"]
speed_data_date_b = df.add_suffix("_b")

In [50]:
speed_data_date_all = pd.merge(speed_data_date_a, speed_data_date_b, left_on='date_id_a', right_on='date_id_b')

In [51]:
# match a and b are the same.

speed_data_date_all[["match_b", "match_a"]]

# drop outcome variable
speed_data_date_all.drop(["match_b"], axis = 1, inplace = True)
speed_data_date_all.rename(columns={"match_a": "match"})

# drop iid columns
speed_data_date_all.drop(["iid_a", "iid_b", "samerace_b"], axis = 1, inplace = True)

In [52]:
def f1(row):
    if row['is_creative_a'] == row['is_creative_b']:
        val = 1
    else:
        val = 0
    return val

speed_data_date_all['both_creative'] = speed_data_date_all.apply(f1, axis=1)

def f2(row):
    if row['is_sporty_a'] == row['is_sporty_b']:
        val = 1
    else:
        val = 0
    return val

speed_data_date_all['both_sporty'] = speed_data_date_all.apply(f2, axis=1)

In [53]:
# age difference
speed_data_date_all["age_difference"] = abs(speed_data_date_all["age_a"] - speed_data_date_all["age_b"])

In [54]:
# difference between what person b i slooking for a what person a rates themselves 1_1_b and 3_1_a

speed_data_date_all["attr_b_a"] = abs(speed_data_date_all["attr1_1_b"] - speed_data_date_all["attr3_1_a"])
speed_data_date_all["sinc_b_a"] = abs(speed_data_date_all["sinc1_1_b"] - speed_data_date_all["sinc3_1_a"])
speed_data_date_all["intel_b_a"] = abs(speed_data_date_all["intel1_1_b"] - speed_data_date_all["intel3_1_a"])
speed_data_date_all["fun_b_a"] = abs(speed_data_date_all["fun1_1_b"] - speed_data_date_all["fun3_1_a"])
speed_data_date_all["amb_b_a"] = abs(speed_data_date_all["amb1_1_b"] - speed_data_date_all["amb3_1_a"])


# difference between what person a i slooking for b what person a rates themselves 1_1_a and 3_1_b
speed_data_date_all["attr_a_b"] = abs(speed_data_date_all["attr1_1_a"] - speed_data_date_all["attr3_1_b"])
speed_data_date_all["sinc_a_b"] = abs(speed_data_date_all["sinc1_1_a"] - speed_data_date_all["sinc3_1_b"])
speed_data_date_all["intel_a_b"] = abs(speed_data_date_all["intel1_1_a"] - speed_data_date_all["intel3_1_b"])
speed_data_date_all["fun_a_b"] = abs(speed_data_date_all["fun1_1_a"] - speed_data_date_all["fun3_1_b"])
speed_data_date_all["amb_a_b"] = abs(speed_data_date_all["amb1_1_a"] - speed_data_date_all["amb3_1_b"])


In [55]:
# difference between hobby preferences
speed_data_date_all["go_out_df"] = abs(speed_data_date_all["go_out_a"] - speed_data_date_all["go_out_b"])
speed_data_date_all["sports_df"] = abs(speed_data_date_all["sports_a"] - speed_data_date_all["sports_b"])
speed_data_date_all["tvsports_df"] = abs(speed_data_date_all["tvsports_a"] - speed_data_date_all["tvsports_b"])
speed_data_date_all["exercise_df"] = abs(speed_data_date_all["exercise_a"] - speed_data_date_all["exercise_b"])
speed_data_date_all["dining_df"] = abs(speed_data_date_all["dining_a"] - speed_data_date_all["dining_b"])
speed_data_date_all["museums_df"] = abs(speed_data_date_all["museums_a"] - speed_data_date_all["museums_b"])
speed_data_date_all["art_df"] = abs(speed_data_date_all["art_a"] - speed_data_date_all["art_b"])
speed_data_date_all["hiking_df"] = abs(speed_data_date_all["hiking_a"] - speed_data_date_all["hiking_b"])
speed_data_date_all["gaming_df"] = abs(speed_data_date_all["gaming_a"] - speed_data_date_all["gaming_b"])
speed_data_date_all["clubbing_df"] = abs(speed_data_date_all["clubbing_a"] - speed_data_date_all["clubbing_b"])

speed_data_date_all["reading_df"] = abs(speed_data_date_all["reading_a"] - speed_data_date_all["reading_b"])
speed_data_date_all["tv_df"] = abs(speed_data_date_all["tv_a"] - speed_data_date_all["tv_b"])
speed_data_date_all["theater_df"] = abs(speed_data_date_all["theater_a"] - speed_data_date_all["theater_b"])
speed_data_date_all["movies_df"] = abs(speed_data_date_all["movies_a"] - speed_data_date_all["movies_b"])
speed_data_date_all["concerts_df"] = abs(speed_data_date_all["concerts_a"] - speed_data_date_all["concerts_b"])
speed_data_date_all["music_df"] = abs(speed_data_date_all["music_a"] - speed_data_date_all["music_b"])
speed_data_date_all["shopping_df"] = abs(speed_data_date_all["shopping_a"] - speed_data_date_all["shopping_b"])
speed_data_date_all["yoga_df"] = abs(speed_data_date_all["yoga_a"] - speed_data_date_all["yoga_b"])
speed_data_date_all["exphappy_df"] = abs(speed_data_date_all["exphappy_a"] - speed_data_date_all["exphappy_b"])


In [56]:
difference_hobbies = ["go_out_df", "sports_df", "tvsports_df", "exercise_df", "dining_df", "museums_df", "art_df", "hiking_df", "gaming_df",
           "clubbing_df", "reading_df", "tv_df", "theater_df", "movies_df", "concerts_df", "music_df", "shopping_df", "yoga_df", "exphappy_df"]

for h in difference_hobbies:
    new_col_b = h + "_big"
    speed_data_date_all[new_col_b] = np.where(speed_data_date_all[h] >= 0.5, 1, 0)

In [57]:
import math
def get_distance(x_1, x_2, y_1, y_2): 

  try:

    x = (float(x_1) - float(x_2))**2
    y = (float(y_1) - float(y_2))**2
    distance = math.sqrt(x + y)

  except:
    distance = -1

  return distance

In [58]:
speed_data_date_all['distance'] = speed_data_date_all.apply(lambda row : get_distance(row["latitude_a"], row["longitude_a"], row["latitude_b"], row["longitude_b"]), axis = 1)

In [59]:
# put distance in bins
speed_data_date_all['distance'] = speed_data_date_all['distance'].astype("float")
bins = np.arange(speed_data_date_all['distance'].min()-1, speed_data_date_all['distance'].max(), 25)
labels = create_labels(bins)
labels[0] = "No zipcode provided"
speed_data_date_all['distance_bin'] = pd.cut(x=speed_data_date_all['distance'],bins = bins, labels = labels)
speed_data_date_all['distance_bin']

0       No zipcode provided
1           123.0 and 148.0
2           198.0 and 223.0
3           123.0 and 148.0
4           148.0 and 173.0
               ...         
2909          73.0 and 98.0
2910          73.0 and 98.0
2911    No zipcode provided
2912          73.0 and 98.0
2913          73.0 and 98.0
Name: distance_bin, Length: 2914, dtype: category
Categories (9, object): ['No zipcode provided' < '23.0 and 48.0' < '48.0 and 73.0' < '73.0 and 98.0' ... '123.0 and 148.0' < '148.0 and 173.0' < '173.0 and 198.0' < '198.0 and 223.0']

In [60]:
speed_data_date_all["zipcode_area_b"] = speed_data_date_all["zipcode_area_b"].replace("Not", 0)
speed_data_date_all["zipcode_area_a"] = speed_data_date_all["zipcode_area_a"].replace("Not", 0)

In [61]:
speed_data_date_all["reading_df"]

0       0.250000
1       0.000000
2       0.000000
3       0.000000
4       0.166667
          ...   
2909    0.500000
2910    0.333333
2911    0.416667
2912    0.583333
2913    0.416667
Name: reading_df, Length: 2914, dtype: float64

In [62]:
speed_data_date_all.to_csv("../data/interim/single_row_date.csv")

In [63]:
# for columns where missing is < 30% then impute missing data

# for columns where missing is > 30%- drop